In [37]:
import math
import numpy as np
import geopandas as gpd
import tensorflow as tf
from tensorflow.keras import layers, applications, callbacks
from sklearn.model_selection import train_test_split
from PIL import Image
from pyproj import Geod
import satellite_imagery_tile_request as sitr

# Configuration
API_KEY = "<API-KEY>"  # Replace with your HERE API key
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 15
ZOOM_LEVEL = 20
TILE_FORMAT = "png"
TILE_SIZE = 512

# --------------------------
# 1. Enhanced Dataset Creation
# --------------------------
def create_dataset(validations_gdf, signs_gdf):
    images = []
    labels = []
    geod = Geod(ellps="WGS84")
    sign_ids = signs_gdf["id"].tolist()
    
    print(f"Processing {len(validations_gdf)} validation entries...")
    
    for idx, row in validations_gdf.iterrows():
        try:
            # Get validation details
            feature_id = row["Feature ID"]
            lat, lon = row.geometry.y, row.geometry.x
            
            # Fetch satellite tile using your module
            sitr.tile_size = TILE_SIZE  # Set global variable
            wkt_polygon = sitr.get_satellite_tile(
                lat, lon, ZOOM_LEVEL, TILE_FORMAT, API_KEY
            )
            
            # Load saved image
            img = Image.open(f"satellite_tile.{TILE_FORMAT}").convert("RGB")
            
            # Check sign existence
            sign_exists = feature_id in sign_ids
            if sign_exists:
                sign_row = signs_gdf[signs_gdf["id"] == feature_id].iloc[0]
                dist = geod.inv(lon, lat, 
                               sign_row.geometry.x, sign_row.geometry.y)[2]
                sign_exists = dist <= 10  # 10m threshold
                
            # Preprocess image
            img = img.resize((IMG_SIZE, IMG_SIZE))
            img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
            
            images.append(img_array)
            labels.append(1 if sign_exists else 0)
            
        except Exception as e:
            print(f"Error processing row {idx}: {str(e)}")
            continue
            
    return np.array(images), np.array(labels)

# --------------------------
# 2. Model Building
# --------------------------
def build_model():
    base_model = applications.EfficientNetB0(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights="imagenet"
    )
    base_model.trainable = False
    
    model = tf.keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(1, activation="sigmoid")
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(name="precision")]
    )
    return model

# --------------------------
# 3. Training Workflow
# --------------------------
def main():
    # Load data
    validations = gpd.read_file("data/validations.geojson")
    signs = gpd.read_file("data/signs.geojson")
    
    # Create dataset
    X, y = create_dataset(validations, signs)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)
    
    # Build and train model
    model = build_model()
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[callbacks.EarlyStopping(patience=3)]
    )
    
    # Save model
    model.save("sign_detection_model.h5")

if __name__ == "__main__":
    main()

Processing 3 validation entries...
Tile saved successfully.
Tile saved successfully.
Tile saved successfully.
Epoch 1/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 0.0000e+00 - loss: 0.7933 - precision: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.3451 - val_precision: 1.0000
Epoch 2/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 1.0000 - loss: 0.3802 - precision: 1.0000 - val_accuracy: 1.0000 - val_loss: 0.1458 - val_precision: 1.0000
Epoch 3/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 1.0000 - loss: 0.1132 - precision: 1.0000 - val_accuracy: 1.0000 - val_loss: 0.0658 - val_precision: 1.0000
Epoch 4/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 1.0000 - loss: 0.0735 - precision: 1.0000 - val_accuracy: 1.0000 - val_loss: 0.0307 - val_precision: 1.0000
Epoch 5/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 1.0000 - loss: 0.0526 - precision: 1.0000 - val_accuracy: 1.0000 - val_loss: 0.0153 - val_precision: 1.0000
Epoch 6/15
1/1 ━━━━━━━━━━━━━━━━━━

In [40]:
def resolve_missing_signs(validation_gdf, confidence_threshold=0.3):
    resolved_signs = []
    
    for _, row in validation_gdf.iterrows():
        try:
            # Get sign coordinates
            lat, lon = row.geometry.y, row.geometry.x
            
            # Fetch satellite tile
            sitr.get_satellite_tile(lat, lon,20,"png",API_KEY)
            
            # Load and preprocess image
            img = Image.open("satellite_tile.png").convert("RGB")
            img = img.resize((224, 224))
            img_array = np.expand_dims(img, axis=0) / 255.0
            
            # Predict
            prediction = model.predict(img_array)[0][0]
            
            # Take action
            if prediction < confidence_threshold:
                resolved_signs.append(row["Feature ID"])
                # Delete sign from HERE database (pseudocode)
                # here_api.delete_sign(feature_id=row["Feature ID"])
                
        except Exception as e:
            print(f"Error processing {row['Feature ID']}: {str(e)}")
    
    return resolved_signs

Tile saved successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 971ms/step
Tile saved successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Tile saved successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Resolved signs: ['urn:here::here:signs:1621752876050240343']


In [41]:
def generate_verification_report(resolved_signs, validations_gdf):
    html = """
    <html>
    <body>
        <h1>Scenario 1 Verification Report</h1>
    """
    
    for sign_id in resolved_signs:
        row = validations_gdf[validations_gdf["Feature ID"] == sign_id].iloc[0]
        html += f"""
        <div style="border: 1px solid #ccc; margin: 10px; padding: 10px;">
            <h3>Sign ID: {sign_id}</h3>
            <p>Coordinates: {row.geometry.y:.6f}, {row.geometry.x:.6f}</p>
            <img src="validation_{sign_id}.png" width="512">
            <p><a href="https://platform.here.com/admin-ui/map?lat={row.geometry.y}&lon={row.geometry.x}&zoom=20" target="_blank">
                Open in HERE Map Viewer
            </a></p>
        </div>
        """
    
    html += "</body></html>"
    
    with open("verification_report.html", "w") as f:
        f.write(html)

# After running resolution
generate_verification_report(missing_signs, validations)